In [2]:
from tensorflow import keras
import numpy as np
import pandas as pd
import PIL
from PIL import Image
import os
from os import path
from keras.utils import np_utils
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [3]:
# Chargement du jeu de données :
def load_signaux(dir):
    img_list = []
    class_counter = 0
    class_list = []
    for i in os.listdir(dir):
        if (path.isdir(path.join(dir, i))):
            for j in os.listdir(path.join(dir, i)):
                if (j.endswith('.ppm')):
                    class_list.append(class_counter)
                    img_list.append(np.array(Image.open(os.path.join(os.path.join(dir, i), j)).resize((28, 28))))
            class_counter += 1
    return np.array(img_list), np.array(class_list)

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
train_dir = "Signaux/train"
test_dir = "Signaux/test"

In [6]:
X_train_full,y_train_full=load_signaux(train_dir)

In [7]:
X_test,y_test = load_signaux(test_dir)

In [8]:
X_train_full.shape

(4575, 28, 28, 3)

In [9]:
y_train_full.shape

(4575,)

In [10]:
X_test.shape

(2520, 28, 28, 3)

In [11]:
y_test.shape

(2520,)

In [12]:
# Normalisation de l'input
X_train_full= X_train_full.astype('float32')
X_test_full= X_test.astype('float32')
X_train_full= X_train_full/ 255.0
X_test_full= X_test/ 255.0

In [13]:
# Categorical => like dummies
y_train_full= np_utils.to_categorical(y_train_full)
y_test_full= np_utils.to_categorical(y_test)


In [14]:
X_train, X_val, y_train, y_val = train_test_split(X_train_full,
                                                  y_train_full,
                                                  test_size=0.25, 
                                                  random_state=42, 
                                                  stratify=y_train_full)

In [15]:
num_classes= y_train_full.shape[1]

# Creation du modele

model = Sequential()
model.add(Conv2D(64, (4, 4), input_shape=(28,28,3), padding='same', activation='relu'))
model.add(Dropout(0.5))
model.add(keras.layers.AveragePooling2D())
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile model
epochs= 20
lrate= 0.001
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        3136      
_________________________________________________________________
dropout (Dropout)            (None, 28, 28, 64)        0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               4

In [167]:
import datetime
Log_dir = "logs/fit/" + datetime.datetime.now().strftime("%m%d-%H%M%S")
Log_dir

'logs/fit/0119-135309'

In [168]:
#Log_dir=os.path.dirname(Log_dir)

callbacks = [
keras.callbacks.TensorBoard(Log_dir),
keras.callbacks.EarlyStopping(patience=5),
keras.callbacks.ModelCheckpoint("signaux_model_"+datetime.datetime.now().strftime("%m%d-%H%M%S")+".h5", save_best_only=True),
]

In [169]:
model.fit(
    X_train, y_train,
    epochs=40,
    validation_data=(X_val, y_val),
    callbacks=callbacks
)

Epoch 1/40
108/108 [==============================] - 7s 64ms/step - loss: 3.5963 - accuracy: 0.1421 - val_loss: 2.2836 - val_accuracy: 0.4449
Epoch 2/40
108/108 [==============================] - 6s 54ms/step - loss: 2.0719 - accuracy: 0.4762 - val_loss: 1.2961 - val_accuracy: 0.6879
Epoch 3/40
108/108 [==============================] - 6s 60ms/step - loss: 1.3928 - accuracy: 0.6061 - val_loss: 0.8242 - val_accuracy: 0.7622
Epoch 4/40
108/108 [==============================] - 6s 57ms/step - loss: 0.9524 - accuracy: 0.7030 - val_loss: 0.5711 - val_accuracy: 0.8383ss: - ETA: 0s - loss: 0.958
Epoch 5/40
108/108 [==============================] - 6s 58ms/step - loss: 0.7103 - accuracy: 0.7782 - val_loss: 0.4362 - val_accuracy: 0.8872
Epoch 6/40
108/108 [==============================] - 7s 65ms/step - loss: 0.4779 - accuracy: 0.8531 - val_loss: 0.3767 - val_accuracy: 0.8899
Epoch 7/40
108/108 [==============================] - 7s 63ms/step - loss: 0.4707 - accuracy: 0.8530 - val_loss: 0.

In [170]:
model.evaluate(X_test_full,y_test_full)

79/79 [==============================] - 1s 7ms/step - loss: 0.2018 - accuracy: 0.9623


[0.20180675387382507, 0.9623016119003296]

In [73]:
%load_ext tensorboard

In [83]:
X_test[0].shape

(28, 28, 3)

In [116]:
Log_dir=os.path.dirname(Log_dir)
Log_dir

'logs/fit'

In [175]:
model  = keras.models.load_model('signaux_model_0119-135310.h5')

In [176]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 28, 28, 64)        3136      
_________________________________________________________________
dropout_20 (Dropout)         (None, 28, 28, 64)        0         
_________________________________________________________________
average_pooling2d_6 (Average (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 14, 14, 32)        18464     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 7, 7, 32)          0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 1568)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 256)             

In [187]:
model.save('model')

INFO:tensorflow:Assets written to: model\assets


In [185]:
%python -v

UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).
